In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import bilby
import gwpy.timeseries

import sys
sys.path.insert(0,'../src')
from util import *
from cross_correlation import *
from postprocessing import *
from pe import *
from constants import *
from simulation import *

In [ ]:
matplotlib.rcParams.update({'font.size':15})

In [ ]:
##################
# Inputs     
##################

NSegments         = 15
Fs                = 1024. # Hz
segmentDuration   = 64.0 # s
t0                = 0 # s
TAvg              = 4.0 # s

####################
# Computed quantities
####################

NSamplesPerSegment=int(segmentDuration*Fs) 
deltaT=1/Fs
fNyquist=1/(2*deltaT)
deltaF=1/segmentDuration
deltaFStoch=1/TAvg
NAvgs = 2 * int(segmentDuration / TAvg) - 1
jobDuration = NSegments * segmentDuration
N = NSegments*NSamplesPerSegment   #Total number of samples

Nd = 2   #Number of detectors

In [ ]:
# Discrete times
T = N*deltaT
t = np.array([t0 + deltaT*i for i in range(int(N))])

if N%2==0:
    numFreqs = N/2-1
else:
    numFreqs = (N-1)/2

#Discrete positive frequencies
deltaF = 1/(N*deltaT)
f = np.array([deltaF*(i+1) for i in range(int(numFreqs))])

In [ ]:
# Generate a noise PSD for each of the detectors (here an example of colored noise, but could use H1/L1 PSD)

noisePSD=np.zeros((Nd,int(numFreqs)))

noisePSD[0]=1e-45*f**(-.5)+1e-45*f**(-.5)*np.random.randn(int(numFreqs))/10
noisePSD[1]=1e-45*f**(-.5)+1e-45*f**(-.5)*np.random.randn(int(numFreqs))/10

In [ ]:
# Sanity check: plotting the noise PSD
plt.loglog(f,noisePSD[0],label='PSD 1')
plt.loglog(f,noisePSD[1],label='PSD 2')
plt.legend()
plt.xlabel('f [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
###################
# Signal properties
###################

fref = 25   #reference frequency (Hz)
alpha = 3  #spectral index
OmegaRef=1e-5 #Amplitude GW spectrum as in h^2Omega=OmegaRef*(f/fref)**alpha

OmegaGW=OmegaRef*(f/fref)**alpha

In [ ]:
# Simulate cross-correlated data
yData=simulate_data(f,noisePSD,OmegaGW)

In [ ]:
# Splice data to prevent issues with periodicity of the ifft
xdata=splice_segments(yData)

In [ ]:
d1=TimeSeries(t,xdata[0])
d2=TimeSeries(t,xdata[1])

In [ ]:
nperseg=int(d1.Fs/deltaF)
freqs,P1_test = welch_psd(d1.data,nperseg=nperseg,window='Hann',fs=d1.Fs)
freqs,P2_test = welch_psd(d2.data,nperseg=nperseg,window='Hann',fs=d2.Fs)

fig0,axs0=plt.subplots()
axs0.loglog(freqs,P1_test,label='Data 1')
axs0.loglog(freqs,P2_test,label='Data 2')
axs0.set_xlim(freqs[0],freqs[-1])
axs0.set_xlabel('f[Hz]')
axs0.set_title('Power spectral density')

## Stochastic CC search

In [ ]:
# run stochastic pipeline
alpha=0 #This alpha simply follows from cross-corelation statistic, don't change even for other spectral alphas
fref=25
Y_t,sig_t,Y_ft,var_ft,segmentStartTimes,freqs=cross_correlation(d1,d2,segmentDuration,deltaFStoch,orf_file='../src/orfs/ORF_HL.dat',
                                                      fref=fref,alpha=alpha,fmin=deltaFStoch,fmax=fNyquist/2.)

In [ ]:
# plot results of CC search
fig,axs=plt.subplots()
axs.loglog(freqs,np.abs(Y_ft[:,0]),label='Y(f)')
axs.loglog(freqs,np.abs(np.sqrt(var_ft[:,0])),label=('sigma(f)'))
#axs.loglog(freqs,freqs**(3)/100000,label=('~f³'))
axs.legend()
axs.set_xlabel('f [Hz]')
axs.set_title('First segment')
plt.show()
# Note that you get the expected f³ dependence for both the CC and sigma (see 3 and 4 in O2). Indeed,if orf=1 and 
# signal is flat, only expect the frequency dependency from S_0(f).

In [ ]:
Y_f,var_f=postprocessing_spectra(Y_ft,var_ft,jobDuration,segmentDuration,
                                 deltaFStoch,deltaT)

In [ ]:
# test optimal filtering

alphas=np.linspace(-8,8,100)

snrs=np.zeros(alphas.shape)
for ii,a in enumerate(alphas):
    y,s=calc_Y_sigma_from_Yf_varf(np.real(Y_f),
                          var_f,
                          freqs=freqs,
                          alpha=a,
                          fref=25)
    snrs[ii]=y/s
fig1,axs1=plt.subplots()
axs1.plot(alphas,snrs)
axs1.axvline(3)
axs1.set_xlabel('alpha')
axs1.set_ylabel('SNR')
axs1.set_title('Optimal SNR vs alpha')
plt.show()

## Parameter estimation

In [ ]:
# Run parameter estimation
label = 'GWB_powerlaw'
outdir = 'outdir'

cleanup_dir(outdir)

Amin,Amax=1e-13,1e-2
    
fref=25

likelihood = BasicPowerLawGWBLikelihood(Y_f[1:],var_f[1:],freqs[1:],fref)
priors = dict(A=bilby.core.prior.LogUniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.Gaussian(0,3.5, 'alpha'))

# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=500,
    walks=10, outdir=outdir, label=label,maxmcmc=10000)
result.plot_corner()

In [ ]:
# extract samples

A1=result.samples[:,0]

x=np.linspace(Amin,Amax,10)

plt.hist(np.log10(A1),bins=30,histtype='step',color='blue',density=True,label='')
plt.axvline(-5,color='red')

plt.xlabel('log$_{10}$ \u03A9$_{ref}$')
plt.ylabel('Posterior probability density')